

# Exercises

For the exams work on at least two of the these three exercises

## Q2: My own test

- Pick a piece of python code that you like (your own PhD project, or take one of the exercises from this class). Implement a unit test and a regression test. 
- Put it under git version control, and write a github action that runs the tests at every commit.
- Edit the github options to make sure the code *cannot* be committed if the tests fail (that's a common thing for big projects with many collaborations, nobody is allowed to break the code)

[repository: xtd, branch: exam](https://github.com/AuroraPerego/xtd/tree/exam)

## Q4: It's over Anakin, I have the high ground. I hate you! 

Pick a piece of your code that your really hate.  You hate it because it's soooooo slooooow. 
- Profile it.
- Find the culprit
- Rewrite that bottleneck making a better use of numpy arrays, or jitting it with Numba.
- Enjoy your faster code

**You underestimate my power**.

In [1]:
import awkward as ak
import numpy as np
import uproot as uproot

In [2]:
C = 29.9792458 #cm/ns

In [3]:
filename = 'histo_SinglePi_24fc_1hit_recTime.root'
file = uproot.open(filename)

In [4]:
file.keys()

['ticlDumper;1',
 'ticlDumper/rechits;11',
 'ticlDumper/rechits;10',
 'ticlDumper/tracksters;1',
 'ticlDumper/clusters;1',
 'ticlDumper/trackstersMerged;1',
 'ticlDumper/associations;1',
 'ticlDumper/simtrackstersSC;1',
 'ticlDumper/simtrackstersCP;1']

In [5]:
simtrackstersSC = file["ticlDumper/simtrackstersSC"]
tracksters  = file["ticlDumper/tracksters"]
associations = file["ticlDumper/associations"]
clusters = file["ticlDumper/clusters"]

In [6]:
cluster_hits_with_time = clusters["cluster_number_of_hits_with_time"].array()

In [7]:
SC_boundx        = simtrackstersSC["boundaryX"].array()
SC_boundy        = simtrackstersSC["boundaryY"].array()
SC_boundz        = simtrackstersSC["boundaryZ"].array()
SC_bx            = simtrackstersSC["barycenter_x"].array()
SC_by            = simtrackstersSC["barycenter_y"].array()
SC_bz            = simtrackstersSC["barycenter_z"].array()
SC_boundary_time = simtrackstersSC["timeBoundary"].array()
SC_CALO_time     = simtrackstersSC["time"].array()
SC_CALO_timeErr  = simtrackstersSC["timeError"].array()
SC_trackIdx      = simtrackstersSC["trackIdx"].array()
SC_vt            = simtrackstersSC["vertices_time"].array()
SC_vtErr         = simtrackstersSC["vertices_timeErr"].array()
SC_vi            = simtrackstersSC["vertices_indexes"].array()
SC_vx            = simtrackstersSC["vertices_x"].array()
SC_vy            = simtrackstersSC["vertices_y"].array()
SC_vz            = simtrackstersSC["vertices_z"].array()

In [8]:
assoc_reco2sim = associations.arrays(["tsCLUE3D_recoToSim_SC_sharedE",
"tsCLUE3D_recoToSim_SC_score",
"tsCLUE3D_recoToSim_SC"])

In [9]:
ts3D = tracksters.arrays(["barycenter_x",
"barycenter_y",
"barycenter_z",
"time",
"timeError",
"vertices_x",
"vertices_y",
"vertices_z",
"vertices_time",
"vertices_indexes"])

## recoTracksters time validation

In [10]:
def distance(x1,y1,z1,x2,y2,z2):
    return ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**0.5

def project_lc_to_pca(point, segment_start, segment_end):
    """
    `lc` is a 1D NumPy array of length 3 representing the point;
    `segment_start` and `segment_end` are 1D NumPy arrays of length 3 representing the start and end points of the line segment. 
    The function returns the minimum distance between the point and the line segment and the closest point on this segment.
    """
    segment_vector = segment_end - segment_start
    #print(segment_vector)
    point_vector = point - segment_start
    #print(point_vector)
    if not np.any(segment_vector):
        projection = 0
    else:
        projection = np.dot(point_vector, segment_vector) / np.dot(segment_vector, segment_vector)

    closest_point = segment_start + projection * segment_vector
    return np.linalg.norm(point - closest_point), closest_point

def computeCaloTime(ev, vx, vy, vz, vt, vi, LChitTime, bx, by, bz, NMIN =3, MR=3):
    n_ts = 0
    time_ts = 0
    for i, t in enumerate(vt):
        nHitsWithTime = LChitTime[vi[i]]
        if t != -99 and project_lc_to_pca(np.array([vx[i], vy[i], vz[i]]), np.array([0.,0.,0.]), np.array([bx, by, bz]))[0] < MR and nHitsWithTime >= NMIN:
            n_ts+=1
            if np.abs(bz)< np.abs(vz[i]):
                time_ts += t - distance(bx, by, bz, vx[i], vy[i], vz[i])/C
            else:
                time_ts += t + distance(bx, by, bz, vx[i], vy[i], vz[i])/C
    if n_ts == 0:
        return -99.
    else:
        return time_ts/n_ts
    return -99.

def compute_residuals_r2s(NMIN = 3):
    residui=[]
    for ev, (recoToSim_index_ev, recoToSim_score_ev, ts_bx_ev, ts_by_ev, ts_bz_ev, ts_vx_ev, ts_vy_ev, ts_vz_ev, ts_vt_ev, ts_vi_ev, LChitTime) in enumerate(zip(assoc_reco2sim.tsCLUE3D_recoToSim_SC, 
    assoc_reco2sim.tsCLUE3D_recoToSim_SC_score, ts3D.barycenter_x, ts3D.barycenter_y, ts3D.barycenter_z, ts3D.vertices_x, ts3D.vertices_y, ts3D.vertices_z, ts3D.vertices_time, 
    ts3D.vertices_indexes, cluster_hits_with_time)):
        for r2s_score, r2s_idx, bx, by, bz, vx, vy, vz, vt, vi in zip(recoToSim_score_ev, recoToSim_index_ev, ts_bx_ev, ts_by_ev, ts_bz_ev, ts_vx_ev, ts_vy_ev, ts_vz_ev, ts_vt_ev, ts_vi_ev):
            t = computeCaloTime(ev, vx, vy, vz, vt, vi, LChitTime, bx, by, bz, NMIN)
            si = r2s_idx[np.argmin(r2s_score)]
            boundt = SC_boundary_time[ev][si]
            if np.min(r2s_score) < 0.35 and boundt != -99 and t != -99:
                boundx = SC_boundx[ev][si]
                boundy = SC_boundy[ev][si]
                boundz = SC_boundz[ev][si]
                deltaT = t - boundt
                deltaS = distance(boundx, boundy, boundz, bx, by, bz)
                results = deltaT - deltaS / C
                residui.append(results)
    return residui

In [11]:
import cProfile, pstats
profiler = cProfile.Profile()
profiler.enable()
res = compute_residuals_r2s()
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

         26118634 function calls (25501428 primitive calls) in 51.060 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   51.060   25.530 /cvmfs/sft.cern.ch/lcg/views/LCG_105_swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3400(run_code)
        2    0.000    0.000   51.060   25.530 {built-in method builtins.exec}
        1    0.000    0.000   51.060   51.060 /tmp/ipykernel_1306/2534407993.py:4(<module>)
        1    0.360    0.360   51.060   51.060 /tmp/ipykernel_1306/1682533672.py:39(compute_residuals_r2s)
   544948    2.644    0.000   34.426    0.000 /cvmfs/sft.cern.ch/lcg/views/LCG_105_swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/awkward/highlevel.py:578(__getitem__)
    18050    0.690    0.000   32.083    0.002 /tmp/ipykernel_1306/1682533672.py:22(computeCaloTime)
1089896/544948    1.369    0.000   27.767    0.000 {built-in method built

In [12]:
!pip install snakeviz

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: gosam 2.1.1-4b98559 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of gosam or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: professor2 X.Y.Z has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of professor2 or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [13]:
%load_ext snakeviz
%snakeviz compute_residuals_r2s()

 
*** Profile stats marshalled to file '/tmp/tmpos3yyj8y'. 
Embedding SnakeViz in this document...
<function display at 0x7fc25e5ca790>


In [14]:
from numba import njit, prange

@njit
def distance_numba(x1,y1,z1,x2,y2,z2):
    return ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**0.5

@njit
def dot_numba(x, y):
    s = 0
    for i in range(len(x)):
        s += x[i]*y[i]
    return s

@njit 
def distance_numba(x1,y1,z1,x2,y2,z2):
    return ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**0.5

@njit
def minNumba(arr):
    if len(arr) == 0:
        raise ValueError("argmin: array is empty")

    min_value = arr[0]

    for i in range(1, len(arr)):
        if arr[i] < min_value:
            min_value = arr[i]

    return min_value

@njit
def project_lc_to_pca_numba(point, segment_start, segment_end):
    """
    `lc` is a 1D NumPy array of length 3 representing the point;
    `segment_start` and `segment_end` are 1D NumPy arrays of length 3 representing the start and end points of the line segment. 
    The function returns the minimum distance between the point and the line segment and the closest point on this segment.
    """
    segment_vector = segment_end - segment_start
    #print(segment_vector)
    point_vector = point - segment_start
    #print(point_vector)
    if not np.any(segment_vector):
        projection = 0
    else:
        projection = dot_numba(point_vector, segment_vector) / dot_numba(segment_vector, segment_vector)

    closest_point = segment_start + projection * segment_vector
    return np.linalg.norm(point - closest_point), closest_point

@njit
def computeCaloTime_numba(ev, vx, vy, vz, vt, vi, LChitTime, bx, by, bz, NMIN =3, MR=3):
    n_ts = 0
    time_ts = 0
    for i, t in enumerate(vt):
        nHitsWithTime = LChitTime[vi[i]]
        if t != -99 and project_lc_to_pca_numba(np.array([vx[i], vy[i], vz[i]]), np.array([0.,0.,0.]), np.array([bx, by, bz]))[0] < MR and nHitsWithTime >= NMIN:
            n_ts+=1
            if np.abs(bz)< np.abs(vz[i]):
                time_ts += t - distance_numba(bx, by, bz, vx[i], vy[i], vz[i])/C
            else:
                time_ts += t + distance_numba(bx, by, bz, vx[i], vy[i], vz[i])/C
    if n_ts == 0:
        return -99.
    else:
        return time_ts/n_ts
    return -99.

def compute_residuals_r2s_v2(NMIN = 3):
    residui=[]
    for ev, (recoToSim_index_ev, recoToSim_score_ev, ts_bx_ev, ts_by_ev, ts_bz_ev, ts_vx_ev, ts_vy_ev, ts_vz_ev, ts_vt_ev, ts_vi_ev, LChitTime) in enumerate(zip(assoc_reco2sim.tsCLUE3D_recoToSim_SC, 
    assoc_reco2sim.tsCLUE3D_recoToSim_SC_score, ts3D.barycenter_x, ts3D.barycenter_y, ts3D.barycenter_z, ts3D.vertices_x, ts3D.vertices_y, ts3D.vertices_z, ts3D.vertices_time, 
    ts3D.vertices_indexes, cluster_hits_with_time)):
        for r2s_score, r2s_idx, bx, by, bz, vx, vy, vz, vt, vi in zip(recoToSim_score_ev, recoToSim_index_ev, ts_bx_ev, ts_by_ev, ts_bz_ev, ts_vx_ev, ts_vy_ev, ts_vz_ev, ts_vt_ev, ts_vi_ev):
            t = computeCaloTime_numba(ev, vx, vy, vz, vt, vi, LChitTime, bx, by, bz, NMIN)
            si = r2s_idx[np.argmin(r2s_score)]
            boundt = SC_boundary_time[ev][si]
            if np.min(r2s_score) < 0.35 and boundt != -99 and t != -99:
                boundx = SC_boundx[ev][si]
                boundy = SC_boundy[ev][si]
                boundz = SC_boundz[ev][si]
                deltaT = t - boundt
                deltaS = distance(boundx, boundy, boundz, bx, by, bz)
                results = deltaT - deltaS / C
                residui.append(results)
    return residui

In [15]:
res2 = compute_residuals_r2s_v2()

In [16]:
import cProfile 
cProfile.run('compute_residuals_r2s_v2()', sort='cumtime')

         39624607 function calls (38451585 primitive calls) in 56.166 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   56.166   56.166 {built-in method builtins.exec}
        1    0.000    0.000   56.166   56.166 <string>:1(<module>)
        1    1.390    1.390   56.166   56.166 1299398470.py:68(compute_residuals_r2s_v2)
   108300    0.282    0.000   28.072    0.000 dispatcher.py:724(typeof_pyval)
398100/108300    0.694    0.000   27.772    0.000 typeof.py:32(typeof)
398100/108300    0.661    0.000   27.471    0.000 functools.py:883(wrapper)
   108300    0.097    0.000   26.980    0.000 __init__.py:64(typeof_Array)
   108300    0.651    0.000   26.884    0.000 highlevel.py:1436(numba_type)
    90750    0.845    0.000   23.139    0.000 arrayview.py:314(fromarray)
181500/90750    0.756    0.000   12.838    0.000 layout.py:52(typeof)
    90750    0.922    0.000   12.736    0.000 layout.py:145(typ

In [17]:
@njit
def argminNumba(arr):
    if len(arr) == 0:
        raise ValueError("argmin: array is empty")

    min_index = 0
    min_value = arr[0]

    for i in range(1, len(arr)):
        if arr[i] < min_value:
            min_value = arr[i]
            min_index = i

    return min_index

In [18]:
@njit
def compute_residuals_r2s_v3(NMIN = 3):
    residui=[]
    for ev, (recoToSim_index_ev, recoToSim_score_ev, ts_bx_ev, ts_by_ev, ts_bz_ev, ts_vx_ev, ts_vy_ev, ts_vz_ev, ts_vt_ev, ts_vi_ev, LChitTime) in enumerate(zip(assoc_reco2sim.tsCLUE3D_recoToSim_SC, 
    assoc_reco2sim.tsCLUE3D_recoToSim_SC_score, ts3D.barycenter_x, ts3D.barycenter_y, ts3D.barycenter_z, ts3D.vertices_x, ts3D.vertices_y, ts3D.vertices_z, ts3D.vertices_time, 
    ts3D.vertices_indexes, cluster_hits_with_time)):
        for r2s_score, r2s_idx, bx, by, bz, vx, vy, vz, vt, vi in zip(recoToSim_score_ev, recoToSim_index_ev, ts_bx_ev, ts_by_ev, ts_bz_ev, ts_vx_ev, ts_vy_ev, ts_vz_ev, ts_vt_ev, ts_vi_ev):
            t = computeCaloTime_numba(ev, vx, vy, vz, vt, vi, LChitTime, bx, by, bz, NMIN)
            si = r2s_idx[argminNumba(r2s_score)]
            boundt = SC_boundary_time[ev][si]
            if minNumba(r2s_score) < 0.35 and boundt != -99 and t != -99:
                boundx = SC_boundx[ev][si]
                boundy = SC_boundy[ev][si]
                boundz = SC_boundz[ev][si]
                deltaT = t - boundt
                deltaS = distance_numba(boundx, boundy, boundz, bx, by, bz)
                results = deltaT - deltaS / C
                residui.append(results)
    return residui

In [19]:
res3 = compute_residuals_r2s_v3()

In [20]:
import cProfile 
cProfile.run('compute_residuals_r2s_v3()', sort='cumtime')

         4 function calls in 0.047 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.047    0.047 {built-in method builtins.exec}
        1    0.000    0.000    0.047    0.047 <string>:1(<module>)
        1    0.047    0.047    0.047    0.047 429975968.py:1(compute_residuals_r2s_v3)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


